In [ ]:
import random
from collections import defaultdict
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import itertools
from sklearn.manifold import TSNE
from sknetwork.data import karate_club, painters, movie_actor
from sknetwork.clustering import Louvain, BiLouvain, modularity, bimodularity
from sknetwork.linalg import normalize
from sknetwork.utils import bipartite2undirected, membership_matrix
from sknetwork.visualization import svg_graph, svg_digraph, svg_bigraph
from sknetwork.hierarchy import LouvainHierarchy, BiLouvainHierarchy
from sknetwork.hierarchy import dasgupta_score, dasgupta_cost
from sklearn.cluster import MiniBatchKMeans, KMeans, FeatureAgglomeration, AffinityPropagation, MeanShift, SpectralClustering, AgglomerativeClustering, AgglomerativeClustering, DBSCAN, Birch

import pickle
import graphviz
from xgboost import plot_tree
import csv
import pandas as pd
import os
import json
import gensim
import pickle
import numpy as np
from pprint import pprint
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from ggplot import *

from scipy.stats import chi2_contingency
from matplotlib import rc

import scipy

import pickle
import shap
import random

import requests
import json
import re
from os.path import isfile, join
from os import listdir
from bs4 import BeautifulSoup
from collections import Counter

from ggplot import *
from sklearn import metrics
from xgboost import XGBClassifier, plot_importance
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    matthews_corrcoef,
    auc,
    average_precision_score,
    fbeta_score
)

from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot
import networkx as nx
import networkx
import community
from yellowbrick.cluster import KElbowVisualizer
from community import community_louvain

import os
import csv
import json
import scipy
import random
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from ggplot import *
from sklearn import metrics
from xgboost import XGBClassifier, plot_importance
from sklearn.metrics import (
    precision_score,
    recall_score,
    accuracy_score,
    f1_score,
    confusion_matrix,
    silhouette_score,
    silhouette_samples,
    calinski_harabasz_score,
    davies_bouldin_score,
    roc_curve,
    matthews_corrcoef,
    auc,
)


In [ ]:
with open('drugId2numId_nod2vec.pkl', 'rb') as f:
    drugId2numId_nod2vec = pickle.load(f)
with open('diseaseId2numId_nod2vec.pkl', 'rb') as f:
    diseaseId2numId_nod2vec = pickle.load(f)
with open('target2name.pkl', 'rb') as f:
    target2name = pickle.load(f)

In [ ]:
disease2name= pd.Series('disease', index=list(diseaseId2numId_nod2vec.values())).to_dict()
drug2name = pd.Series('drug', index=list(drugId2numId_nod2vec.values())).to_dict()

In [ ]:
dict_id = {**drug2name, **disease2name, **target2name}
pd.DataFrame(dict_id.items(), columns=['id', 'type'])['type'].value_counts()

In [ ]:
with open('DTI.pkl', 'rb') as f:
    DTI = pickle.load(f)

In [ ]:
with open('all_nod2vec.pkl', 'rb') as f:
    all_nod2vec = pickle.load(f)

In [ ]:
with open('all_nod2vec_new.pkl', 'rb') as f:
    all_nod2vec_new = pickle.load(f)

In [ ]:
mapping = False
df = all_nod2vec.copy()

######    ######    ######    ######    ######    ######    ######    ######    ######    ######    ######    ######

df = df[df['weight']!=0]
df = df.drop(columns= ['Type_Interaction'])

os.chdir("nod2vec")
df.to_csv('edglist_all.edgelist', sep=' ', index=False, header=False)
os.chdir("EEG_pathway_Embedding")

if mapping:
    # nod2vec (for drug)
    os.system(f'PYTHONHASHSEED=10 python2 node2vec/src/main.py --workers 8 --input dim10_all_plus.emb --weighted --dimensions 10')

embeddings_all = pd.read_csv('dim10_all_plus.emb', sep=' ', skiprows=[0], header=None, index_col=0)  

In [ ]:
embeddings_all_id = embeddings_all.copy()
embeddings_all_id['type']= embeddings_all.index.map(dict_id)

pca = PCA(n_components=2)
pca_result = pca.fit_transform(embeddings_all_id.drop(columns=['type']).values)


embeddings_all_id['PCA-1'] = pca_result[:,0]
embeddings_all_id['PCA-2'] = pca_result[:,1]

chart = ggplot(embeddings_all_id, aes(x='PCA-1', y='PCA-2',  color='factor(type)') ) \
    + geom_point(size=120, alpha=0.8) 

In [ ]:
# removing nodes that we do not have embedding vectors
new_DTI = DTI[DTI['to'].isin(list(embeddings_all.index))]
new_DTI = new_DTI[new_DTI['from'].isin(list(embeddings_all.index))]

In [ ]:
new_DTI['label'] = new_DTI[['S1','S2','S3','S4','S5','S6']].values.tolist()
selected_type = ['increases^expression','decreases^expression','decreases^reaction','increases^reaction', 'increases^activity','decreases^activity', 'negative_DTI']
for i in range (0, len(new_DTI)):
    new_DTI['label'].iloc[i] = list(set(new_DTI['label'].iloc[i])& set(selected_type))
    
print (new_DTI['Type_Interaction'].value_counts())

new_DTI = new_DTI[['from','to', 'label']]

In [ ]:
def concating(df, embedding):
    """
    This function concats drugs and targets vectors
    
    Args: 
        df: A DataFrame of drug-target interactions
        embedding: A DataFrame of embedded vectors for each drug and target
        
    Returns: A DataFrame of drug-target interaction vectors
    """
    dataset = []
    for idx, row in df.iterrows():
        from_vector = embedding.loc[row['from']]
        to_vector = embedding.loc[row['to']]
        features = from_vector.append(to_vector).reset_index(drop=True)
        features = features.append(row)
        dataset.append(features)

    df_final = pd.DataFrame(dataset)
    df_final.drop(columns=['from', 'to'], inplace=True)
    return df_final

In [ ]:
df_final = concating(new_DTI, embeddings_all)

In [ ]:
## create matrix of labels as 0 and 1 (one hot encoder)
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

df_label = pd.DataFrame(mlb.fit_transform(df_final['label']), columns=mlb.classes_, index=df_final.index)
df_label

In [ ]:
## add new label format to gene/drug interaction dataframe
df_interaction_newLabel = pd.concat([df_final.drop(columns=['label']), df_label], axis=1).drop_duplicates()
df_interaction_newLabel

In [ ]:
df_interaction_newLabel_PAC = df_interaction_newLabel.drop(columns= ['increases^expression','decreases^expression','decreases^reaction','increases^reaction', 'increases^activity','decreases^activity'])
df_interaction_newLabel_PAC['type'] = df_interaction_newLabel_PAC['negative_DTI']
df_interaction_newLabel_PAC = df_interaction_newLabel_PAC.drop(columns=['negative_DTI']).drop_duplicates()

pca = PCA(n_components=2)
pca_result = pca.fit_transform(df_interaction_newLabel_PAC.drop(columns=['type']).values)


df_interaction_newLabel_PAC['PCA-1'] = pca_result[:,0]
df_interaction_newLabel_PAC['PCA-2'] = pca_result[:,1]

chart = ggplot(df_interaction_newLabel_PAC, aes(x='PCA-1', y='PCA-2',  color='factor(type)') ) \
    + geom_point(size=120, alpha=0.8) 

chart

In [ ]:
# Report
for i in selected_type:
    n_label = df_interaction_newLabel[i].sum()
    n_rest = len(df_interaction_newLabel)- n_label
    print (f'number of {i}: {n_label} VS. rest: {n_rest}')

In [ ]:
## seperate features (as x) and labels (as y)
def get_data(df_train, label, all_labels):
    df_interaction_new = df_train.copy()
    df_interaction_new['label'] = df_interaction_new[label]
    df_interaction_new.drop(columns=list(all_labels), inplace=True)
    print(df_interaction_new['label'].value_counts())
    
    X = df_interaction_new.drop(columns=['label'])
    y = pd.DataFrame(df_interaction_new['label'])
    
    return X, y.values.ravel()

def get_sample_weight(y_train):
    weight_ratio = float(len(y_train[y_train == 0]))/float(len(y_train[y_train == 1]))
    
    return weight_ratio